    # Client

Demo of a client interacting with a remote agent. 

You can interact with this via API directly

In [ ]:
import requests

inputs = {"input": {"input": "testing"}}
response = requests.post("http://localhost:8000/chat/invoke", json=inputs)

response.json()

You can also interact with this via the RemoteRunnable interface (to use in other chains)

In [24]:
from langserve import RemoteRunnable

remote_runnable = RemoteRunnable("http://localhost:8000/chat/")

Remote runnable has the same interface as local runnables

In [25]:
await remote_runnable.ainvoke({"input": "hi!"})

HTTPStatusError: Client error '422 Unprocessable Entity' for url 'http://localhost:8000/chat/invoke'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/422 for {"detail":[{"loc":["messages"],"msg":"field required","type":"value_error.missing"}]}

In [ ]:
from langchain_core.messages import HumanMessage

remote_runnable.invoke(HumanMessage(content="Good evening!"))

## Stream

Please note that streaming alternates between actions and observations. It does not stream individual tokens! If you need to stream individual tokens you will need to use astream_log!

In [ ]:
async for chunk in remote_runnable.astream(

        HumanMessage(content="Good evening!")):
    print('--')
    print(chunk)

## Stream Events

The client is looking for a runnable name called `agent` for the chain events. This name was defined on the server side using `runnable.with_config({"run_name": "agent"}`

In [26]:
inputs = {"messages": [HumanMessage(content="Good evening!")]}

async for event in remote_runnable.astream_events(inputs, {"em_model": "oai-gpt4"}, version="v1"):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
                event["name"] == "main_agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('inputs')}"
            )
    elif kind == "on_chain_end":
        if (
                event["name"] == "main_agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    # if kind == "on_chain_stream":
    #     content = event["data"]["chunk"].content
    #     if content:
    #         # Empty content in the context of OpenAI means
    #         # that the model is asking for a tool to be invoked.
    #         # So we only print non-empty content
    #         print(content, end="|")
    # elif kind == "on_tool_start":
    #     print("--")
    #     print(
    #         f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
    #     )
    # elif kind == "on_tool_end":
    #     print(f"Done tool: {event['name']}")
    #     print(f"Tool output was: {event['data'].get('output')}")
    #     print("--")

Starting agent: main_agent with input: None

--


KeyError: 'output'

## Stream log

If you need acccess the individual llm tokens from an agent use `astream_log`. Please make sure that you set **streaming=True** on your LLM (see server code). For this to work, the LLM must also support streaming!

In [27]:
inputs = {"messages": [HumanMessage(content="Good evening!")]}

async for chunk in remote_runnable.astream_events(inputs, {"em_model": "oai-gpt4"}, version="v1", ):
    print('--')
    print(chunk)

--
{'event': 'on_chain_start', 'run_id': '21424389-e4a6-40f7-9547-134a4bbbadab', 'name': '/chat', 'tags': [], 'metadata': {'em_model': 'oai-gpt4'}, 'data': {'input': {'messages': [HumanMessage(content='Good evening!')]}}, 'parent_ids': []}
--
{'event': 'on_chain_start', 'name': '__start__', 'run_id': 'b71c5b84-67aa-4b35-ac10-2e137031aa78', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {'em_model': 'oai-gpt4', 'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_task_idx': 0, 'thread_ts': '1ef5c3ee-1e0f-6002-bffe-731fe2b322dc'}, 'data': {'input': {'messages': [HumanMessage(content='Good evening!')]}}, 'parent_ids': []}
--
{'event': 'on_chain_end', 'name': '__start__', 'run_id': 'b71c5b84-67aa-4b35-ac10-2e137031aa78', 'tags': ['graph:step:0', 'langsmith:hidden'], 'metadata': {'em_model': 'oai-gpt4', 'langgraph_step': 0, 'langgraph_node': '__start__', 'langgraph_triggers': ['__start__'], 'langgraph_task_idx': 0, 'thread_ts': '1ef5

In [23]:
import requests

response = requests.post(
    "http://localhost:8000/chat/stream",
    json={'input': {'messages': [{"role": "human", "content": "hi"}]}}
)
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)